# Data Loading to Dataframe

The first step is to load the Find Journal Lines document into a dataframe.

In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
trans_detail = pd.read_excel("Find Journal Lines - NMSS 2020-12-10 11_34 PST.xlsx", header = 36)

Wall time: 12.4 s


In [3]:
print(trans_detail.head())

                                             Journal Journal Number  \
0  Operational Journal: FF Fast Forward, LLC. - 1...            NaN   
1  Operational Journal: FF Fast Forward, LLC. - 1...            NaN   
2  Operational Journal: FF Fast Forward, LLC. - 1...            NaN   
3  Operational Journal: FF Fast Forward, LLC. - 1...            NaN   
4  Operational Journal: FF Fast Forward, LLC. - 1...            NaN   

  Journal Sequence Number                        Journal Memo  \
0     SUP-2021-Oct-000001  FF-1711-29347 Hodgetts 10_20 26097   
1     SUP-2021-Oct-000001  FF-1711-29347 Hodgetts 10_20 26097   
2     SUP-2021-Oct-000002  FF-1612-26373 Shandler 10_20 28385   
3     SUP-2021-Oct-000002  FF-1612-26373 Shandler 10_20 28385   
4     SUP-2021-Nov-000001       FF-1601-07453 Liu 01_18 24552   

       Operational Transaction   Originated by Approved by  \
0  Supplier Invoice: IN-022668  Vinton Douglas         NaN   
1  Supplier Invoice: IN-022668  Vinton Douglas         NaN

# Formatting

The next step is to add columns to the data so that I can format the way I will need to in the pivot tables.  Currently there is no "Months" column, and not all of the Spend Categories show up in our reports.  Additionally, I need to subtract credits from debits to figure out what hits the reports, and I need to exclude In-Kind from all of our reporting. 

In [4]:
FY20_Data = trans_detail

FY20_Data = FY20_Data.reset_index()

FY20_Data = FY20_Data.drop('index',1)

FY20_Data.rename(columns=FY20_Data.iloc[1])

#print(FY20_Data.iloc[0])

#print(FY20_Data.head())

column_names = ['Journal', 'Journal Number', 'Journal Sequence Number', 'Journal Memo', 'Operational Transaction', 'Originated by', 'Approved by', 'Company',
'Intercompany Initiating Company','Status', 'Accounting Date','Posted Date','Source', 'Ledger','Currency','Ledger Account','Ledger Debit Amount',
'Ledger Credit Amount','Line Memo','External Reference ID', 'Spend Category','Revenue Category', 'Cost Center','Fund',
'Gift','Program','Project','Region','Location','Fundraising Type', 'InKind','State','Bank Account','Intercompany Affiliate','Supplier','Customer','Employee','Match ID']

FY20_Data.columns = column_names

FY20_Data = FY20_Data.drop([0])

FY20_Data = FY20_Data.reset_index()

FY20_Data = FY20_Data.drop('index',1)



In [5]:
FY20_Data['Debit-Credit'] = FY20_Data.apply(lambda x: x['Ledger Debit Amount'] - x['Ledger Credit Amount'], axis=1)
FY20_Data['Accounting Date'] = pd.to_datetime(FY20_Data['Accounting Date'])
FY20_Data['Month - Yr'] = FY20_Data['Accounting Date'].apply(lambda x: x.strftime('%m-%y'))
GL_Lookup = pd.read_excel('Ledger_Account_Table.xlsx', sheet_name = 'Sheet1')
FY20_Data = pd.merge(FY20_Data, GL_Lookup, on='Ledger Account', how='left')

In [6]:
def return_first(x):
    first = x[0]
    return first

In [7]:
FY20_Data['First'] = FY20_Data['Ledger Account'].apply(return_first)

In [8]:
FY20_Data['Exp/DBD/Rev'] = FY20_Data['First'].map({'4':'Revenue', '5':'Expense', '6':'DBD', '1':'BS', '2':'BS', '3':'BS'})

In [9]:
del FY20_Data['First']

In [10]:
FY20_Data['Debit-Credit'].fillna(0, inplace=True)

FY20_Data['Supplier'].fillna("Not Available", inplace=True)

FY20_Data['Line Memo'].fillna("No Description Available", inplace=True)

FY20_Data['Operational Transaction'].fillna("Not Available", inplace=True)

FY20_Data['Journal Memo'].fillna("Not Available", inplace=True)

FY20_Data['Project'].fillna("No Project", inplace=True)

FY20_Data['Spend Category'].fillna("No Spend Category", inplace=True)

FY20_Data['Location'].fillna("No Location", inplace=True)

FY20_Data['Employee'].fillna("No Employee", inplace=True)

FY20_Data['Program'].fillna("No Program", inplace=True)

FY20_Data['Status'].fillna("NA", inplace=True)

FY20_Data['InKind'].fillna("No", inplace=True)

In [11]:
FY20_DataCreatedPosted = FY20_Data[(FY20_Data['Status']=='Posted')|(FY20_Data['Status']=='Created')]

FY20_DataCreatedPosted.shape


(30676, 42)

In [12]:
FY20_Data.columns

Index(['Journal', 'Journal Number', 'Journal Sequence Number', 'Journal Memo',
       'Operational Transaction', 'Originated by', 'Approved by', 'Company',
       'Intercompany Initiating Company', 'Status', 'Accounting Date',
       'Posted Date', 'Source', 'Ledger', 'Currency', 'Ledger Account',
       'Ledger Debit Amount', 'Ledger Credit Amount', 'Line Memo',
       'External Reference ID', 'Spend Category', 'Revenue Category',
       'Cost Center', 'Fund', 'Gift', 'Program', 'Project', 'Region',
       'Location', 'Fundraising Type', 'InKind', 'State', 'Bank Account',
       'Intercompany Affiliate', 'Supplier', 'Customer', 'Employee',
       'Match ID', 'Debit-Credit', 'Month - Yr', 'In/Out', 'Exp/DBD/Rev'],
      dtype='object')

In [13]:
Cost_Centers = list(FY20_Data['Cost Center'].unique())


print(Cost_Centers)

[nan, 'CC001 Research', 'CC005 Facilities', 'CC014 Finance Operations', 'CC009 Marketing', 'CC010 Digital Marketing', 'CC022 Leadership Events', 'CC018 Direct Marketing\xa0', 'CC029 Professional Education', 'CC023 Bike MS', 'CC028 MS Navigator', 'CC008 Legal and Risk', 'CC016 Individual Giving', 'CC033 Community Leadership', 'CC021 Challenge Walk', 'CC012 Technology', 'CC004 Operations', 'CC027 Programs & Services', 'CC007 Volunteer Engagement', 'CC002 Advocacy', 'CC015 Development', 'CC003 ELT', 'CC013 Strategic Finance', 'CC006 Human Resources', 'CC011 Brand Management', 'CC017 Tools and Resources', 'CC032 Services Resources', 'CC025 Other Special Events', 'CC024 Walk MS', 'CC019 Corporate Relations', 'CC020 Do It Yourself (DIY)', 'CC026 MuckFest MS', 'CC034 Community Engagement', 'CC035 Event Production']


# Data Filtering

Now that the dataframe has all the columns I need, the next step is to filter it so that I have a dataframe for each Cost Center and each different group that needs a different report.  That is what I am doing in the next section. 

In [14]:
def createCC(Cost_Center):
    return FY20_DataCreatedPosted[(FY20_DataCreatedPosted['Cost Center']==Cost_Center) & (FY20_DataCreatedPosted['In/Out']=='In') & (FY20_DataCreatedPosted['Exp/DBD/Rev']=='Expense') & (FY20_DataCreatedPosted['InKind']=='No')]
 
    

In [15]:
def create_Occupancy(Cost_Center):
        return FY20_DataCreatedPosted[(FY20_DataCreatedPosted['Cost Center']==Cost_Center) & (FY20_DataCreatedPosted['In/Out']=='In') & (FY20_DataCreatedPosted['Exp/DBD/Rev']=='Expense') & (FY20_DataCreatedPosted['InKind']=='No') & (FY20_DataCreatedPosted['Spend Category']=='Office Cleaning & CAM Expense')|(FY20_DataCreatedPosted['Spend Category']=='Office Moving Expense')|(FY20_DataCreatedPosted['Spend Category']=='Office Rent Expense')|(FY20_DataCreatedPosted['Spend Category']=='Property Tax Expense')|(FY20_DataCreatedPosted['Spend Category']=='Security Expense')|(FY20_DataCreatedPosted['Spend Category']=='Staff Relocation Expense')|(FY20_DataCreatedPosted['Spend Category']=='Storage Rent Expense')|(FY20_DataCreatedPosted['Spend Category']=='Utilities')]

In [16]:
Occupancy = create_Occupancy('CC005 Facilities')

In [17]:
FinanceOps = createCC('CC014 Finance Operations')
StrategicFin = createCC('CC013 Strategic Finance')

Finance = pd.concat([FinanceOps, StrategicFin], ignore_index=True, sort=False)


In [18]:
Ops = createCC('CC004 Operations')
ELT = createCC('CC003 ELT')

Operations = pd.concat([Ops, ELT], ignore_index=True, sort=False)

In [19]:
HR = createCC('CC006 Human Resources')
IndividualGiving = createCC('CC016 Individual Giving')
VE = createCC('CC007 Volunteer Engagement')
Legal = createCC('CC008 Legal and Risk')
Professional_Education = createCC('CC029 Professional Education')
Corporate_Relations = createCC('CC019 Corporate Relations')
Direct_Marketing = createCC('CC018 Direct Marketing\xa0')
Marketing = createCC('CC009 Marketing')
Research = createCC('CC001 Research')
MS_Navigator = createCC('CC028 MS Navigator')
Development = createCC('CC015 Development')
Digital_Marketing = createCC('CC010 Digital Marketing')
Community_Engagement = createCC('CC034 Community Engagement')
Community_Leadership = createCC('CC033 Community Leadership')
Event_Production = createCC('CC035 Event Production')
Tools_and_Resources = createCC('CC017 Tools and Resources')
Brand_Management = createCC('CC011 Brand Management')
Fundraising_Support = createCC('CC036 Fundraising Support')
Facilities = createCC('CC005 Facilities')
Advocacy = createCC('CC002 Advocacy')
ProgramsandServices = createCC('CC027 Programs & Services')
Services_Resources = createCC('CC032 Services Resources')


In [20]:
DevelopmentRollup = pd.concat([Development, IndividualGiving, Tools_and_Resources, Event_Production, Fundraising_Support, Corporate_Relations], ignore_index=True, sort=False)

In [21]:
CommunityRollup = pd.concat([Community_Engagement, Community_Leadership], ignore_index=True, sort=False)

In [22]:
ProgramsandServicesRollup = pd.concat([ProgramsandServices, MS_Navigator, Professional_Education, Services_Resources], ignore_index=True, sort=False)

In [23]:
MarketingRollup = pd.concat([Marketing, Digital_Marketing, Brand_Management], ignore_index=True, sort=False)

In [24]:
Technology = createCC('CC012 Technology')

In [25]:
Occupancy["Spend Category"].value_counts()

Office Rent Expense              352
Storage Rent Expense             203
Office Cleaning & CAM Expense    162
Utilities                         76
Property Tax Expense              64
Office Moving Expense             10
Security Expense                   8
Name: Spend Category, dtype: int64

# Create Pivot Tables

In this section, I am creating the Pivot Tables in a way where we can match up the totals with what shows up in our reports from Workday, but so that we can also see all of the detail that we would nee to see to understand these expenses in the Pivot Tables.  When I bring in "Operational Transaction", we will be given the Supplier Invoices numbers which we can use to search Workday for whatever Supplier Invoices we would need.

In [26]:
def create_project_pivot(Group):
    Month_order = ['10-20', '11-20', '12-20', '01-21', '02-21', '03-21', '04-21', '05-21', '06-21', '07-21', '08-21', '09-21']
    table = pd.pivot_table(Group, index = ['Project','Spend Category','Supplier','Line Memo', 'Journal Memo', 'Operational Transaction', 'Employee'], values = 'Debit-Credit', columns = 'Month - Yr', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [27]:
def create_Rollup_pivot(Group):
    Month_order = ['10-20', '11-20', '12-20', '01-21', '02-21', '03-21', '04-21', '05-21', '06-21', '07-21', '08-21', '09-21']
    table = pd.pivot_table(Group, index = ['Cost Center','Spend Category','Supplier','Line Memo', 'Journal Memo', 'Operational Transaction', 'Employee'], values = 'Debit-Credit', columns = 'Month - Yr', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [28]:
def create_Occupancy_pivot(Group):
    Month_order = ['10-20', '11-20', '12-20', '01-21', '02-21', '03-21', '04-21', '05-21', '06-21', '07-21', '08-21', '09-21']
    table = pd.pivot_table(Group, index = ['Location','Spend Category','Supplier','Line Memo', 'Journal Memo', 'Operational Transaction'], values = 'Debit-Credit', columns = 'Month - Yr', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [29]:
def create_Services_pivot(Group):
    Month_order = ['10-20', '11-20', '12-20', '01-21', '02-21', '03-21', '04-21', '05-21', '06-21', '07-21', '08-21', '09-21']
    table = pd.pivot_table(Group, index = ['Program','Spend Category','Supplier','Line Memo', 'Journal Memo', 'Operational Transaction', 'Employee'], values = 'Debit-Credit', columns = 'Month - Yr', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [30]:
Occupancypivot = create_Occupancy_pivot(Occupancy)

In [31]:
ProgramsandServicespivot = create_Services_pivot(ProgramsandServices)

In [32]:
Technologypivot = create_project_pivot(Technology)

In [33]:
Marketingpivot = create_project_pivot(Marketing)

In [34]:
def create_basic_pivot(Group):
    Month_order = ['10-20', '11-20', '12-20', '01-21', '02-21', '03-21', '04-21', '05-21', '06-21', '07-21', '08-21', '09-21']
    table = pd.pivot_table(Group, index = ['Spend Category','Supplier','Line Memo', 'Journal Memo', 'Operational Transaction', 'Employee'], values = 'Debit-Credit', columns = 'Month - Yr', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [35]:
import os

path = r'C:\Users\jeremy.wendt\Anaconda3\envs\pandas_trans_FY21\Details'

In [36]:
Financepivot = create_basic_pivot(Finance)
VEpivot = create_basic_pivot(VE)
HRpivot = create_basic_pivot(HR)
Legalpivot = create_basic_pivot(Legal)
Prof_Edpivot = create_basic_pivot(Professional_Education)
DirectMarketingpivot = create_basic_pivot(Direct_Marketing)
Researchpivot = create_basic_pivot(Research)
MS_Navigatorpivot = create_basic_pivot(MS_Navigator)
Developmentpivot = create_basic_pivot(Development)
Digital_Marketingpivot = create_basic_pivot(Digital_Marketing)
Community_Engagementpivot = create_basic_pivot(Community_Engagement)
Community_Leadershippivot = create_basic_pivot(Community_Leadership)
Event_Productionpivot = create_basic_pivot(Event_Production)
ToolsandResourcespivot = create_basic_pivot(Tools_and_Resources)
Brand_Managementpivot = create_basic_pivot(Brand_Management)
#FundraisingSupportpivot = create_basic_pivot(Fundraising_Support)
Facilitiespivot = create_basic_pivot(Facilities)
Advocacypivot = create_basic_pivot(Advocacy)
Services_Resourcespivot = create_basic_pivot(Services_Resources)
Operationspivot = create_basic_pivot(Operations)


In [37]:
DevelopmentRolluppivot = create_Rollup_pivot(DevelopmentRollup)
CommunityRolluppivot = create_Rollup_pivot(CommunityRollup)
MarketingRolluppivot = create_Rollup_pivot(MarketingRollup)
ProgramsandServicesRolluppivot = create_Rollup_pivot(ProgramsandServicesRollup)

# Add Subtotals

Adding subtotals to Pandas Pivot Tables is not intuitive or easy, but it is very important so that the user can tie the totals to what we see in the reports out of Workday.

In [38]:
def add_subtotal(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Spend Category')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [39]:
def add_subtotal_Project(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Project')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [40]:
def add_subtotal_Rollup(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Cost Center')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [41]:
def add_subtotal_Occupancy(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Location')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [42]:
def add_subtotal_Programs(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Program')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [43]:
Occupancypivot = add_subtotal_Occupancy(Occupancypivot)

In [44]:
DevelopmentRolluppivot = add_subtotal_Rollup(DevelopmentRolluppivot)
CommunityRolluppivot = add_subtotal_Rollup(CommunityRolluppivot)
MarketingRolluppivot = add_subtotal_Rollup(MarketingRolluppivot)
ProgramsandServicesRolluppivot = add_subtotal_Rollup(ProgramsandServicesRolluppivot)

In [45]:
Marketingpivot = add_subtotal_Project(Marketingpivot)
Technologypivot = add_subtotal_Project(Technologypivot)

In [46]:
ProgramsandServicespivot = add_subtotal_Programs(ProgramsandServicespivot)

In [47]:
Financepiv = add_subtotal(Financepivot)
HRpivot = add_subtotal(HRpivot)
VEpivot = add_subtotal(VEpivot)
Legalpivot = add_subtotal(Legalpivot)
Prof_Edpivot = add_subtotal(Prof_Edpivot)
DirectMarketingpivot = add_subtotal(DirectMarketingpivot)
Researchpivot = add_subtotal(Researchpivot)
MS_Navigatorpivot = add_subtotal(MS_Navigatorpivot)
Developmentpivot = add_subtotal(Developmentpivot)
Digital_Marketingpivot = add_subtotal(Digital_Marketingpivot)
Community_Engagementpivot = add_subtotal(Community_Engagementpivot)
Community_Leadershippivot = add_subtotal(Community_Leadershippivot)
Event_Productionpivot = add_subtotal(Event_Productionpivot)
ToolsandResourcespivot = add_subtotal(ToolsandResourcespivot)
Brand_Managementpivot = add_subtotal(Brand_Managementpivot)
#FundraisingSupportpivot = add_subtotal(FundraisingSupportpivot)
Facilitiespivot = add_subtotal(Facilitiespivot)
Advocacypivot = add_subtotal(Advocacypivot)
Services_Resourcespivot = add_subtotal(Services_Resourcespivot)
Operationspivot = add_subtotal(Operationspivot)

# Export Pivot Table to Excel File

Using Pandas to_excel() function, I can freeze panes and export the pivot table to an excel spreadsheet, but I am not able to format it with Pandas.  For that, I need to use Openpyxl.  For now, I'm creating the Pivot Tables with pandas in Excel files to come back to and format with Openpyxl in the next step.

In [48]:
def to_excel(pivot, str):
    pivot.to_excel(path+str, freeze_panes = (1,6), engine = 'openpyxl', sheet_name = 'Trans Detail', float_format = "%.0f")

In [49]:
def to_excel_Project(pivot, str):
    pivot.to_excel(path+str, freeze_panes = (1,7), engine = 'openpyxl', sheet_name = 'Trans Detail', float_format = "%.0f")

In [50]:
to_excel_Project(DevelopmentRolluppivot,'\DevelopmentRollup_TransDetail.xlsx')
to_excel_Project(CommunityRolluppivot,'\CommunityRollup_TransDetail.xlsx')
to_excel_Project(MarketingRolluppivot,'\MarketingRollup_TransDetail.xlsx')
to_excel_Project(ProgramsandServicesRolluppivot,'\ProgramsandServicesRollup_TransDetail.xlsx')

In [51]:
to_excel_Project(Technologypivot,'\Technology_TransDetail.xlsx')
to_excel_Project(Marketingpivot,'\Marketing_TransDetail.xlsx')
to_excel_Project(ProgramsandServicespivot,'\ProgramsandServices_TransDetail.xlsx')

In [52]:
to_excel(Occupancypivot,'\Occupancy_TransDetail.xlsx')

In [53]:

file_names = ['\Advocacy_TransDetail.xlsx', '\ToolsandResources_TransDetail.xlsx', '\Operations_TransDetail.xlsx',  '\Community_Leadership_TransDetail.xlsx', '\Facilities_TransDetail.xlsx', '\Services_Resources_TransDetail.xlsx', '\Brand_Management_TransDetail.xlsx', '\VE_TransDetail.xlsx', '\HR_TransDetail.xlsx', '\Finance_TransDetail.xlsx', '\Legal_TransDetail.xlsx', '\Prof_Ed_TransDetail.xlsx', '\DirectMarketing_TransDetail.xlsx', '\Research_TransDetail.xlsx', '\MS_Navigator_TransDetail.xlsx', '\Development_TransDetail.xlsx', '\Digital_Marketing_TransDetail.xlsx', '\Community_Engagement_TransDetail.xlsx', '\Event_Production_TransDetail.xlsx'] 

to_excel(VEpivot,'\VE_TransDetail.xlsx')
to_excel(Financepiv,'\Finance_TransDetail.xlsx')
to_excel(Legalpivot,'\Legal_TransDetail.xlsx')
to_excel(HRpivot,'\HR_TransDetail.xlsx')
to_excel(Prof_Edpivot,'\Prof_Ed_TransDetail.xlsx')
to_excel(DirectMarketingpivot,'\DirectMarketing_TransDetail.xlsx')
to_excel(Researchpivot,'\Research_TransDetail.xlsx')
to_excel(MS_Navigatorpivot,'\MS_Navigator_TransDetail.xlsx')
to_excel(Developmentpivot,'\Development_TransDetail.xlsx')
to_excel(Digital_Marketingpivot,'\Digital_Marketing_TransDetail.xlsx')
to_excel(Community_Engagementpivot,'\Community_Engagement_TransDetail.xlsx')
to_excel(Event_Productionpivot,'\Event_Production_TransDetail.xlsx')
to_excel(ToolsandResourcespivot,'\ToolsandResources_TransDetail.xlsx')
to_excel(Brand_Managementpivot,'\Brand_Management_TransDetail.xlsx')
#to_excel(FundraisingSupportpivot,'\FundraisingSupport_TransDetail.xlsx')
to_excel(Facilitiespivot,'\Facilities_TransDetail.xlsx')
to_excel(Advocacypivot,'\Advocacy_TransDetail.xlsx')
to_excel(Services_Resourcespivot,'\Services_Resources_TransDetail.xlsx')
to_excel(Operationspivot,'\Operations_TransDetail.xlsx')
to_excel(Community_Leadershippivot,'\Community_Leadership_TransDetail.xlsx')







#  Openpyxl formatting

In this section, I am taking the excel file that I already saved with Pandas, and formatting it for the end user.  This includes adding column labels, number formatting, bold formatting, color formatting for the headers, changing column widths, changing alignment, and adding borders.  Finally, after this I am saving the files over the old excel files.

In [54]:
from openpyxl.styles import Color, PatternFill, Font, Border
greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')

In [55]:
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side


def edit_workbook(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Spend Category"
    sheet['B1']= "Supplier"
    sheet['C1']= "Line Memo"
    sheet['D1']= "Journal Memo"
    sheet['E1']= "Operational Transaction"
    sheet['F1']= "Employee"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 12.5
    sheet.column_dimensions['I'].width = 12.5
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(7, 22):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 7):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=6).value == "Subtotal":
            for j in range(7,20):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    book.save(file)

In [56]:
def edit_workbook_projects(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Project"
    sheet['B1']= "Spend Category"
    sheet['C1']= "Supplier"
    sheet['D1']= "Line Memo"
    sheet['E1']= "Journal Memo"
    sheet['F1']= "Operational Transaction"
    sheet['G1']= "Employee"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 20
    sheet.column_dimensions['F'].width = 15 
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 12.5
    sheet.column_dimensions['I'].width = 12.5
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5       
    sheet.column_dimensions['E'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(8, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 8):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill            
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=7).value == "Subtotal":
            for j in range(8,21):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    book.save(file)

In [57]:
def edit_workbook_programs(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Program"
    sheet['B1']= "Spend Category"
    sheet['C1']= "Supplier"
    sheet['D1']= "Line Memo"
    sheet['E1']= "Journal Memo"
    sheet['F1']= "Operational Transaction"
    sheet['G1']= "Employee"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 20
    sheet.column_dimensions['F'].width = 15 
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 12.5
    sheet.column_dimensions['I'].width = 12.5
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5     
    sheet.column_dimensions['E'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(8, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 8):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill            
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=7).value == "Subtotal":
            for j in range(8,21):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    book.save(file)

In [58]:

def edit_workbook_Occupancy(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Location"
    sheet['B1']= "Spend Category"
    sheet['C1']= "Supplier"
    sheet['D1']= "Line Memo"
    sheet['E1']= "Journal Memo"
    sheet['F1']= "Operational Transaction"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 20
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 12.5
    sheet.column_dimensions['H'].width = 12.5
    sheet.column_dimensions['I'].width = 12.5
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5         
    sheet.column_dimensions['E'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(7, 22):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 7):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill  
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=6).value == "Subtotal":
            for j in range(7,20):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))         
    book.save(file)

In [59]:
def edit_workbook_CC(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Cost Center"
    sheet['B1']= "Spend Category"
    sheet['C1']= "Supplier"
    sheet['D1']= "Line Memo"
    sheet['E1']= "Journal Memo"
    sheet['F1']= "Operational Transaction"
    sheet['G1']= "Employee"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 20
    sheet.column_dimensions['F'].width = 15 
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 12.5
    sheet.column_dimensions['I'].width = 12.5
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5     
    sheet.column_dimensions['E'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(8, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 8):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill            
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=7).value == "Subtotal":
            for j in range(8,21):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    book.save(file)

In [60]:
edit_workbook_projects(path+'\Technology_TransDetail.xlsx')
edit_workbook_projects(path+'\Marketing_TransDetail.xlsx')
edit_workbook_programs(path+'\ProgramsandServices_TransDetail.xlsx')



In [61]:
to_excel_Project(DevelopmentRolluppivot,'\DevelopmentRollup_TransDetail.xlsx')
to_excel_Project(CommunityRolluppivot,'\CommunityRollup_TransDetail.xlsx')
to_excel_Project(MarketingRolluppivot,'\MarketingRollup_TransDetail.xlsx')
to_excel_Project(ProgramsandServicesRolluppivot,'\ProgramsandServicesRollup_TransDetail.xlsx')

In [62]:
edit_workbook_CC(path+'\DevelopmentRollup_TransDetail.xlsx')
edit_workbook_CC(path+'\CommunityRollup_TransDetail.xlsx')
edit_workbook_CC(path+'\MarketingRollup_TransDetail.xlsx')
edit_workbook_CC(path+'\ProgramsandServicesRollup_TransDetail.xlsx')

In [63]:
edit_workbook_Occupancy(path+'\Occupancy_TransDetail.xlsx')

In [64]:
%%time
for i in file_names:
    edit_workbook(path+i)

Wall time: 5.75 s
